Vamos a extraer las features del report con los datos de cada replay

In [1]:
import pandas as pd
import ast

file_path = '../../data/report.csv'
fecha = '05-13'
labels_report = '../../data/'+ fecha + '-labels-constructs.csv'

df = pd.read_csv(file_path)
contextFeatures = df["contextFeatures"].astype('str')
contextFeatures = contextFeatures.apply(lambda x: ast.literal_eval(x))
contextFeatures = contextFeatures.apply(pd.Series)
usedFigures = contextFeatures["UsedFigures"].astype('str')
usedFigures = usedFigures.apply(lambda x: ast.literal_eval(x))
usedFigures = usedFigures.apply(pd.Series)
df = df.drop(columns=["contextFeatures"])
contextFeatures = contextFeatures.drop(columns=["UsedFigures"])
df = pd.concat([df, contextFeatures, usedFigures], axis=1)
df.head()


,replay,user,group,puzzle,globalAttemptId,attemptId,attemptFeatures,ActiveTime,InactiveTime,InteractionEvents,...,Snapshots,TimeStampSnapshots,CheckSolution,Completed,cube,pyramid,ramp,cylinder,cone,sphere
0,2. Separated Boxes~1,2d3db94690a19a62a0942fbd6ac30308,4fe25833f555e9903d2bb6bbeec3fbfb,2. Separated Boxes,2,"{'#Attempt': 1, '#GlobalAttempt': 2, 'ActiveTi...",NaN,27.355295,0.0,4,...,0,[],1,True,2,0,0,0,0,0
1,3. Rotate a Pyramid~1,2d3db94690a19a62a0942fbd6ac30308,4fe25833f555e9903d2bb6bbeec3fbfb,3. Rotate a Pyramid,3,"{'#Attempt': 1, '#GlobalAttempt': 3, 'ActiveTi...",NaN,164.386631,0.0,46,...,1,[163.0],3,False,0,14,0,0,0,0
2,3. Rotate a Pyramid~2,2d3db94690a19a62a0942fbd6ac30308,4fe25833f555e9903d2bb6bbeec3fbfb,3. Rotate a Pyramid,5,"{'#Attempt': 2, '#GlobalAttempt': 5, 'ActiveTi...",NaN,170.763438,0.0,49,...,0,[],5,True,0,1,0,0,0,0
3,Bear Market~1,2d3db94690a19a62a0942fbd6ac30308,4fe25833f555e9903d2bb6bbeec3fbfb,Bear Market,4,"{'#Attempt': 1, '#GlobalAttempt': 4, 'ActiveTi...",NaN,40.556214,0.0,7,...,0,[],1,False,0,0,0,0,0,6
4,4. Match Silhouettes~1,2d3db94690a19a62a0942fbd6ac30308,4fe25833f555e9903d2bb6bbeec3fbfb,4. Match Silhouettes,6,"{'#Attempt': 1, '#GlobalAttempt': 6, 'ActiveTi...",NaN,100.489310,0.0,66,...,0,[],1,True,1,5,0,0,0,0


Como solo usamos replays completadas filtramos los datos por la columna `completed` y eliminamos la columna `completed` que ya no aporta información relevante.
Quitamos también las columnas que no vamos a usar.

In [2]:
df = df.drop(columns=["globalAttemptId","attemptId","attemptFeatures","InteractionEvents","TimeStampSnapshots"])
df_completed = df.loc[df["Completed"]==True]
df_completed = df_completed.drop(columns=["Completed"])
df_completed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4416 entries, 0 to 6994
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   replay         4416 non-null   object 
 1   user           4416 non-null   object 
 2   group          4416 non-null   object 
 3   puzzle         4416 non-null   object 
 4   ActiveTime     4416 non-null   float64
 5   InactiveTime   4416 non-null   float64
 6   CreateShape    4416 non-null   int64  
 7   RotateShape    4416 non-null   int64  
 8   MoveShape      4416 non-null   int64  
 9   ScaleShape     4416 non-null   int64  
 10  DeleteFigure   4416 non-null   int64  
 11  RotateView     4416 non-null   int64  
 12  Snapshots      4416 non-null   int64  
 13  CheckSolution  4416 non-null   int64  
 14  cube           4416 non-null   int64  
 15  pyramid        4416 non-null   int64  
 16  ramp           4416 non-null   int64  
 17  cylinder       4416 non-null   int64  
 18  cone         

Ahora vamos a usar las columnas `replay` y `user` como id para unir los datos de los report con los de los replays.

In [3]:
df_sr = pd.read_csv(labels_report)
df_sr = df_sr.rename(columns={"User": "user","Replay":"replay"})
df_sr.head()

,user,replay,spatial_reasoning
0,00f86c5fc28e8d326cdae17ea97c7387,45-Degree Rotations~1,3.375000
1,00f86c5fc28e8d326cdae17ea97c7387,Angled Silhouette~1,2.666667
2,00f86c5fc28e8d326cdae17ea97c7387,Bird Fez~2,3.541667
3,00f86c5fc28e8d326cdae17ea97c7387,Pi Henge~1,3.166667
4,00f86c5fc28e8d326cdae17ea97c7387,Pyramids are Strange~1,2.500000


In [4]:
merged_df = df_completed.merge(df_sr, on=["user", "replay"])
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 0 to 196
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   replay             197 non-null    object 
 1   user               197 non-null    object 
 2   group              197 non-null    object 
 3   puzzle             197 non-null    object 
 4   ActiveTime         197 non-null    float64
 5   InactiveTime       197 non-null    float64
 6   CreateShape        197 non-null    int64  
 7   RotateShape        197 non-null    int64  
 8   MoveShape          197 non-null    int64  
 9   ScaleShape         197 non-null    int64  
 10  DeleteFigure       197 non-null    int64  
 11  RotateView         197 non-null    int64  
 12  Snapshots          197 non-null    int64  
 13  CheckSolution      197 non-null    int64  
 14  cube               197 non-null    int64  
 15  pyramid            197 non-null    int64  
 16  ramp               197 non

In [5]:
# Comprobamos que aparezcan todas las features que queremos
merged_df.columns

Index(['replay', 'user', 'group', 'puzzle', 'ActiveTime', 'InactiveTime',
       'CreateShape', 'RotateShape', 'MoveShape', 'ScaleShape', 'DeleteFigure',
       'RotateView', 'Snapshots', 'CheckSolution', 'cube', 'pyramid', 'ramp',
       'cylinder', 'cone', 'sphere', 'spatial_reasoning'],
      dtype='object')

In [6]:
# Generamos el .csv que usaremos para los modelos
fname = fecha + "-dataset.csv"
merged_df.to_csv(fname,index=False)